In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [11]:
import pandas as pd
df =pd.read_csv("/home/yadagiri/Downloads/GYANDEEP 2ND PAPER CODE/kaggletestdata1.csv")
df

,text,label
0,California is the biggest example of city plan...,1
1,A life without cars sure sounds like a great i...,0
2,Cars are used used in everyday life but they m...,0
3,The electoral collage should be disbanded. Not...,0
4,"Dear Mr.Mrs. Senator, The Electoral College is...",0
...,...,...
2489,"Dear Florida senator, It is in the interest of...",0
2490,Driving can be a huge hassle. Having to spend ...,0
2491,"Dear\nSenator,\n\nI am writing to express my o...",1
2492,"The electoral college is pointless, I believe ...",0


In [12]:
df.shape

(2494, 2)

In [6]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Load dataset
dataset_path = "/home/yadagiri/Downloads/GYANDEEP 2ND PAPER CODE/kaggletestdata1.csv"
df = pd.read_csv(dataset_path)

X = list(df['text'])
#y = np.asarray(d1[d1.columns[1:]])
y_data = df[df.columns[1:2]]
y = y_data.values
######### Label Encoding for y_train & y_test################
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y
# Split dataset into train and validation sets
train_X, val_X, train_y, val_y = train_test_split(X,y, test_size=0.2, random_state=42)

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # import pdb;pdb.set_trace()
        text = self.texts[idx]
        label = self.labels[idx]  # Convert label to integer
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # Assuming 2 classes for binary classification
batch_size = 32
epochs = 3
# Create DataLoader for train and validation sets
train_dataset = CustomDataset(train_X, train_y, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = CustomDataset(val_X, val_y, tokenizer)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define optimizer and loss function
# optimizer = AdamW(model.parameters(), lr=learning_rate)
# criterion = torch.nn.CrossEntropyLoss()

# Create DataLoader for train and validation sets
# train_dataset = CustomDataset(train_df['text'], train_df['binary_label'], tokenizer)
# val_dataset = CustomDataset(val_df['text'], val_df['binary_label'], tokenizer)

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define training parameters

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = AdamW(model.parameters(), lr=2e-5)



/home/yadagiri/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internall

In [7]:

y = y.flatten()


In [8]:
y

array([1, 0, 0, ..., 1, 0, 0])

In [9]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=learning_rate)


NameError: name 'learning_rate' is not defined

In [10]:
from sklearn.metrics import precision_score, recall_score, f1_score
# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Initialize empty lists to store loss and accuracy values
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
train_precisions = []
val_precisions = []
train_recalls = []
val_recalls = []
train_f1_scores = []
val_f1_scores = []

for epoch in range(epochs):
    model.train()
    epoch_train_loss = 0
    correct_train_preds = 0
    total_train_preds = 0
    train_predicted_labels = []
    train_true_labels = []
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        epoch_train_loss += loss.item()
        _, predicted = torch.max(outputs.logits, 1)
        correct_train_preds += (predicted == labels).sum().item()
        total_train_preds += labels.size(0)
        train_predicted_labels.extend(predicted.cpu().numpy())
        train_true_labels.extend(labels.cpu().numpy())

    # Calculate training accuracy and loss for the epoch
    train_loss = epoch_train_loss / len(train_loader)
    train_accuracy = correct_train_preds / total_train_preds
    train_precision = precision_score(train_true_labels, train_predicted_labels, average='weighted')
    train_recall = recall_score(train_true_labels, train_predicted_labels, average='weighted')
    train_f1 = f1_score(train_true_labels, train_predicted_labels, average='weighted')

    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    train_precisions.append(train_precision)
    train_recalls.append(train_recall)
    train_f1_scores.append(train_f1)

    # Validation loop
    model.eval()
    epoch_val_loss = 0
    correct_val_preds = 0
    total_val_preds = 0
    val_predicted_labels = []
    val_true_labels = []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            epoch_val_loss += loss.item()
            _, predicted = torch.max(outputs.logits, 1)
            correct_val_preds += (predicted == labels).sum().item()
            total_val_preds += labels.size(0)
            val_predicted_labels.extend(predicted.cpu().numpy())
            val_true_labels.extend(labels.cpu().numpy())

    # Calculate validation accuracy and loss for the epoch
    val_loss = epoch_val_loss / len(val_loader)
    val_accuracy = correct_val_preds / total_val_preds
    val_precision = precision_score(val_true_labels, val_predicted_labels, average='weighted')
    val_recall = recall_score(val_true_labels, val_predicted_labels, average='weighted')
    val_f1 = f1_score(val_true_labels, val_predicted_labels, average='weighted')

    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    val_precisions.append(val_precision)
    val_recalls.append(val_recall)
    val_f1_scores.append(val_f1)

    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss}, Train Accuracy: {train_accuracy}, Train Precision: {train_precision}, Train Recall: {train_recall}, Train F1: {train_f1}")
    print(f"Val Loss: {val_loss}, Val Accuracy: {val_accuracy}, Val Precision: {val_precision}, Val Recall: {val_recall}, Val F1: {val_f1}")


Epoch 1/3, Train Loss: 0.44716575883683707, Train Accuracy: 0.7899749373433584, Train Precision: 0.7933771104935861, Train Recall: 0.7899749373433584, Train F1: 0.7873289379332968
Val Loss: 0.3019978357478976, Val Accuracy: 0.8637274549098196, Val Precision: 0.8646797569185412, Val Recall: 0.8637274549098196, Val F1: 0.8639803125897617
Epoch 2/3, Train Loss: 0.2111232087370895, Train Accuracy: 0.9223057644110275, Train Precision: 0.9225691978479906, Train Recall: 0.9223057644110275, Train F1: 0.9221493456324918
Val Loss: 0.3548200121149421, Val Accuracy: 0.8617234468937875, Val Precision: 0.8806230410035034, Val Recall: 0.8617234468937875, Val F1: 0.8621202615032922
Epoch 3/3, Train Loss: 0.10847886441837228, Train Accuracy: 0.9664160401002506, Train Precision: 0.9664250985420071, Train Recall: 0.9664160401002506, Train F1: 0.966400697271761
Val Loss: 0.248824889305979, Val Accuracy: 0.9038076152304609, Val Precision: 0.9092037060866439, Val Recall: 0.9038076152304609, Val F1: 0.902561

In [10]:
from transformers import BertForSequenceClassification, BertTokenizer

# Initialize the model and tokenizer (assuming you already have them)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Save the model and tokenizer
model.save_pretrained("/home/yadagiri/bert_model")
tokenizer.save_pretrained("/home/yadagiri/bert_model")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('/home/yadagiri/bert_model/tokenizer_config.json',
 '/home/yadagiri/bert_model/special_tokens_map.json',
 '/home/yadagiri/bert_model/vocab.txt',
 '/home/yadagiri/bert_model/added_tokens.json')

In [8]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Calculate average test loss and accuracy
avg_test_loss = test_loss / len(test_loader)
test_accuracy = correct_preds / total_preds

# Calculate precision, recall, and F1 score
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

# Print metrics
print(f"Test Loss: {avg_test_loss}, Test Accuracy: {test_accuracy}")
print(f"Precision: {precision}, Recall: {recall}, F1 Score: {f1}")

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)


NameError: name 'test_loader' is not defined